Строим запрос к нашему сервису

In [90]:
import requests

import urllib.request
import json 

In [100]:
# Пример данных
data = (X_test)
print(data)

# формируем запрос
def send_json(x):
#     gender, tenure, PhoneService, TotalCharges, StreamingMovies, StreamingTV, TechSupport = x
    gender, tenure, PhoneService, TotalCharges, StreamingMovies, StreamingTV, TechSupport = x
    #print(description, company_profile, benefits)
    body = {
    "gender": [gender],
    "tenure": [tenure],
    "PhoneService": [PhoneService],
    "TotalCharges": [TotalCharges],
    "StreamingMovies":[StreamingMovies], 
    "StreamingTV":[StreamingTV],
    "TechSupport":[TechSupport]
    }
    myurl = 'http://127.0.0.1:5000/'+'predict'
    headers = {'content-type': 'application/json; charset=utf-8'}
    response = requests.post(myurl, json=body, headers=headers)
    return response.json()['predictions']

# обращение к серверу с запросом из одного набора (его построили руками выше - data)
if __name__ == '__main__':
    response = send_json(data)
    print('предсказание',response)

      gender  tenure  PhoneService  TotalCharges  StreamingMovies  \
0     Female       1             0         24.80                0   
1       Male      41             1        996.45                0   
2     Female      52             1       1031.70                0   
3     Female       1             1         76.35                0   
4       Male      67             1       3260.10                0   
...      ...     ...           ...           ...              ...   
2320    Male      71             1       1901.00                0   
2321    Male      29             0       1636.95                1   
2322    Male       7             1        631.85                0   
2323    Male      32             1       3145.90                1   
2324  Female      59             1       5597.65                0   

      StreamingTV  TechSupport  
0               0            0  
1               0            0  
2               0            0  
3               0            0  
4     

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

Сделаем обработку массы запросов:

1. загрузим данные X_test

In [101]:
import pandas as pd


In [102]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [103]:
# path = '/content/gdrive/MyDrive/2_ver_actual Артем Зраев. 10.2020/Lection9/'
path2='/home/morrowbord/jupiyer-notebook/ML_business/project_churn/'

In [104]:
import pandas as pd
from sklearn.metrics import roc_auc_score,roc_curve,scorer
from urllib import request, parse

X_test = pd.read_csv(path2+"X_test.csv")
y_test = pd.read_csv(path2+"y_test.csv")

2. Передаем запрос одиночный из наших данных и посмотрим на ответ

In [105]:
# response = send_json(X_test[["gender",
#     "tenure",
#     "PhoneService",
#     "TotalCharges",
#     "StreamingMovies", 
#     "StreamingTV",
#     "TechSupport"]].iloc[0,:])

In [106]:
response = send_json(X_test[["gender",
    "tenure",
    "PhoneService",
    "TotalCharges",
    "StreamingMovies", 
    "StreamingTV",
    "TechSupport"]].iloc[0,:])

TypeError: Object of type int64 is not JSON serializable

In [107]:
respons

NameError: name 'respons' is not defined

In [108]:
X_test[['description', 'company_profile', 
                      'benefits']].head()

KeyError: "None of [Index(['description', 'company_profile', 'benefits'], dtype='object')] are in the [columns]"

3. Сделаем N запросов и оценим время

In [12]:
N = 50

In [13]:
%%time
predictions = X_test[['description', 'company_profile', 
                      'benefits']].iloc[:N,:].apply(lambda x: send_json(x),1)

CPU times: user 126 ms, sys: 14.1 ms, total: 140 ms
Wall time: 16.8 s


In [ ]:
predictions.values[:5]

array([0.02617387, 0.04457579, 0.00480416, 0.0010568 , 0.00122864])

4. Посчитаем метрику roc-auc

In [14]:
y_test.head()

,fraudulent
0,0
1,0
2,0
3,0
4,0


In [15]:
from sklearn.metrics import roc_auc_score,roc_curve,scorer

In [16]:
roc_auc_score(y_score=predictions.values, y_true=y_test.values[:N])

0.9270833333333333

5. Анализ : в целом похоже